In [145]:

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
stop_words = set(stopwords.words('english')) 

In [171]:
def seperate_sentences(data):
    return sent_tokenize(data) 

In [205]:
def Sentence_to_POS(all_sentences):

    data=[]
    for sentence in all_sentences: 
        one_sentence_pos = []
        '''
        Word tokenizers is used to find the words and punctuation in a string 
        '''  
        wordsList = nltk.word_tokenize(sentence) 

        '''
        Removing stop words from wordList.
        ''' 
        wordsList = [w for w in wordsList if not w in stop_words]  

        '''
        Using a Tagger. Which is part-of-speech tagger or POS-tagger.  
        '''

        tagged = nltk.pos_tag(wordsList) 
        for x in tagged:
            one_sentence_pos.append(x[1])
        data.append(one_sentence_pos)

    return data

In [208]:
import os
all_files = os.listdir('dataset/three_author_dataset')
novel_POS=[]
novel_label=[]
for filename in all_files:
    full_path = 'dataset/three_author_dataset/'+filename
    with open(full_path, 'r') as f:
        data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
        all_sentences = seperate_sentences(data)
        sentences_to_pos = Sentence_to_POS(all_sentences)
        novel_POS.append(sentences_to_pos)
        novel_label.append(filename[8])  